In [1]:
!sudo apt-get -y -qq update


In [2]:
!sudo apt-get -y -qq install postgresql


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../0-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../1-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../2-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Selecting previously unselected pac

In [5]:
!sudo service postgresql start


 * Starting PostgreSQL 10 database server
   ...done.


In [6]:
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"


ALTER ROLE


In [7]:
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS futurex;'


NOTICE:  database "futurex" does not exist, skipping
DROP DATABASE


In [8]:
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE futurex;'


CREATE DATABASE


In [9]:
!PGPASSWORD=postgres psql -q -h localhost -p 5432 -U postgres -d futurex -f create_model_table.sql


psql:create_model_table.sql:1: NOTICE:  table "futurex_model_catalog" does not exist, skipping


In [10]:
import psycopg2


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [11]:
connection = psycopg2.connect(user='postgres',
                              password='postgres',
                              host='localhost',
                              database='futurex')

In [12]:
cursor = connection.cursor()


In [13]:
select_query = "select * from futurex_model_catalog"


In [14]:
cursor.execute(select_query)


In [15]:
models = cursor.fetchall()


In [16]:
models

[]

In [17]:
import numpy as np
import pandas as pd
training_data = pd.read_csv('storepurchasedata.csv')
X = training_data.iloc[:, :-1].values
y = training_data.iloc[:,-1].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
from sklearn.neighbors import KNeighborsClassifier
# minkowski is for ecledian distance
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [18]:
y_pred = classifier.predict(X_test)
y_prob = classifier.predict_proba(X_test)[:,1]

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test,y_pred))

0.875


In [19]:
import pickle

pickle_classifier_string = pickle.dumps(classifier)
pickle_sc_string = pickle.dumps(sc)

In [20]:
model_insert_sql = "INSERT INTO futurex_model_catalog VALUES(%s, %s, %s)"


In [21]:
insert_tuple = (1, 'classifier', pickle_classifier_string)


In [22]:
cursor = connection.cursor()


In [23]:
cursor.execute(model_insert_sql, insert_tuple)


In [24]:
insert_tuple = (2, 'sc', pickle_sc_string)


In [25]:
cursor.execute(model_insert_sql, insert_tuple)


In [26]:

cursor.close()
connection.commit()

In [27]:
cursor = connection.cursor()
select_query = "select * from futurex_model_catalog"
models =cursor.execute(select_query)
models = cursor.fetchall()
models


[(1, 'classifier', <memory at 0x7f86b216d530>),
 (2, 'sc', <memory at 0x7f86b216d460>)]

In [28]:
classifier_from_db = pickle.loads(models[0][2])
scaler_from_db = pickle.loads(models[1][2])

In [29]:
new_pred = classifier_from_db.predict(scaler_from_db.transform(np.array([[40,20000]])))

new_pred_proba = classifier_from_db.predict_proba(scaler_from_db.transform(np.array([[40,20000]])))[:,1]
print(new_pred)
print(new_pred_proba)

[0]
[0.2]


In [30]:
new_pred = classifier_from_db.predict(scaler_from_db.transform(np.array([[42,50000]])))

new_pred_proba = classifier_from_db.predict_proba(scaler_from_db.transform(np.array([[42,50000]])))[:,1]
print(new_pred)
print(new_pred_proba)

[1]
[0.8]
